In [1]:
import numpy as np
import pandas as pd
import scipy

import string
import re
import gc

In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [37]:
train = pd.read_csv('data/train.csv', encoding='utf-8')
train.dropna(inplace=True)
train = train[train.sentence_id != 492799]
train.reset_index(inplace=True)

#### Prepare full data

In [71]:
vectorizer = CountVectorizer (
    analyzer='char_wb',
    ngram_range=(1, 2),
    lowercase=False
)

In [73]:
%time vectorizer.fit(train['before'].values)

CPU times: user 2min 17s, sys: 5.31 s, total: 2min 23s
Wall time: 2min 24s


CountVectorizer(analyzer='char_wb', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [90]:
len(vectorizer.get_feature_names())

16351

In [77]:
labels = np.array(train['after'].values != train['before'].values, dtype=np.uint8)

enc = LabelEncoder()
labels = enc.fit_transform(labels)

#### Prepare context data

In [91]:
def get_context_value(values, sentences, offset):
    offset_values = np.asarray([u'' for v in range(len(values))], dtype=object)
    for i in range(len(values)):
        offset_value = ""
        i_offset = i + offset
        if i_offset >= 0 and i_offset < len(values) \
                and sentences[i_offset] == sentences[i]:
            offset_value = values[i_offset]
        offset_values[i] = offset_value

    return offset_values

In [40]:
train['before_m_2'] = pd.Series(get_context_value(train.before.values, train.sentence_id.values, -2))
train['before_m_1'] = pd.Series(get_context_value(train.before.values, train.sentence_id.values, -1))
train['before_p_1'] = pd.Series(get_context_value(train.before.values, train.sentence_id.values, 1))
train['before_p_2'] = pd.Series(get_context_value(train.before.values, train.sentence_id.values, 2))

In [93]:
def generate_context_ngrams(data, columns):
    return scipy.sparse.hstack([vectorizer.transform(data[c].values) for c in columns])

In [43]:
context_cols = ['before_m_2', 'before_m_1', 'before', 'before_p_1', 'before_p_2']

#### Prepare train data

In [79]:
sample_sizes = {
    "PUNCT" : 25000,
    "PLAIN" : 30000,
    "CARDINAL" : 20000,
    "LETTERS" : 20000,
    "DATE" : 20000,
    "ORDINAL" : 15000,
    "MEASURE" : 15000
}

# train.reset_index(inplace=True)
train_data = pd.DataFrame(columns=train.columns)
sample_max_size = 10000

for c in train['class'].unique():
    class_sample = train[train['class'] == c]
    sample_size = sample_sizes[c] if c in sample_sizes else sample_max_size
    if len(class_sample) > sample_max_size:
        train_data = train_data.append(class_sample.sample(sample_size))
    else:
        train_data = train_data.append(class_sample)

In [94]:
train_ngrams = generate_context_ngrams(train_data, context_cols)

In [96]:
labels_train = np.array(train_data['after'].values != train_data['before'].values, dtype=np.uint8)
labels_train = enc.transform(labels_train)

### Train XGBoost classifier 

In [97]:
xtr, xcv, ytr, ycv = train_test_split(train_ngrams, labels_train, test_size = 0.05, random_state = 42)
feature_names = ['f_{}'.format(i) for i in range(train_ngrams.shape[1])]

In [98]:
dtrain = xgb.DMatrix(xtr, label=ytr, feature_names=feature_names)
dvalid = xgb.DMatrix(xcv, label=ycv, feature_names=feature_names)

In [99]:
params = [("objective", "binary:logistic"),
          ("booster", "gbtree"),
          ("nthread", 3),
          ("eta", 0.01),
          ("max_depth", 8),
          ("subsample", 0.9),
          ("min_child_weight", 1),
          ("colsample_bytree", 0.7),
          ("eval_metric", 'error'),
          ("eval_metric", 'auc'),
         ]
num_rounds = 10000
stop = 60

In [100]:
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

In [101]:
gbm = xgb.train(params, dtrain, num_rounds, evals=watchlist, early_stopping_rounds=stop)

[0]	train-error:0.15299	train-auc:0.926252	valid-error:0.15659	valid-auc:0.923645
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 60 rounds.
[1]	train-error:0.094578	train-auc:0.969127	valid-error:0.095706	valid-auc:0.968482
[2]	train-error:0.08131	train-auc:0.973787	valid-error:0.080859	valid-auc:0.973352
[3]	train-error:0.086724	train-auc:0.973908	valid-error:0.087161	valid-auc:0.9735
[4]	train-error:0.081333	train-auc:0.973835	valid-error:0.080966	valid-auc:0.973216
[5]	train-error:0.075115	train-auc:0.979218	valid-error:0.076266	valid-auc:0.97869
[6]	train-error:0.065344	train-auc:0.982773	valid-error:0.065798	valid-auc:0.982741
[7]	train-error:0.068492	train-auc:0.984472	valid-error:0.068789	valid-auc:0.984588
[8]	train-error:0.068526	train-auc:0.986457	valid-error:0.069109	valid-auc:0.986652
[9]	train-error:0.067267	train-auc:0.987555	valid-error:0.067614	valid-auc:0.987935
[10]	train-error:0.0652

[96]	train-error:0.055832	train-auc:0.990762	valid-error:0.055757	valid-auc:0.990693
[97]	train-error:0.055854	train-auc:0.990764	valid-error:0.055757	valid-auc:0.990688
[98]	train-error:0.055843	train-auc:0.99148	valid-error:0.055757	valid-auc:0.991638
[99]	train-error:0.055792	train-auc:0.991467	valid-error:0.055757	valid-auc:0.991639
[100]	train-error:0.055815	train-auc:0.991471	valid-error:0.055757	valid-auc:0.991638
[101]	train-error:0.055804	train-auc:0.991474	valid-error:0.055757	valid-auc:0.991651
[102]	train-error:0.05582	train-auc:0.991477	valid-error:0.055864	valid-auc:0.991666
[103]	train-error:0.05582	train-auc:0.992308	valid-error:0.055864	valid-auc:0.992477
[104]	train-error:0.055849	train-auc:0.992314	valid-error:0.055864	valid-auc:0.992495
[105]	train-error:0.055815	train-auc:0.992314	valid-error:0.055864	valid-auc:0.992475
[106]	train-error:0.055854	train-auc:0.992316	valid-error:0.055864	valid-auc:0.992494
[107]	train-error:0.055849	train-auc:0.992781	valid-error:0.0

[192]	train-error:0.030145	train-auc:0.996692	valid-error:0.029801	valid-auc:0.99673
[193]	train-error:0.027008	train-auc:0.996873	valid-error:0.025208	valid-auc:0.99699
[194]	train-error:0.026991	train-auc:0.996875	valid-error:0.025208	valid-auc:0.996987
[195]	train-error:0.031466	train-auc:0.996872	valid-error:0.030015	valid-auc:0.996983
[196]	train-error:0.029729	train-auc:0.997033	valid-error:0.028626	valid-auc:0.997069
[197]	train-error:0.02965	train-auc:0.997038	valid-error:0.028626	valid-auc:0.997058
[198]	train-error:0.029684	train-auc:0.997047	valid-error:0.028733	valid-auc:0.997069
[199]	train-error:0.027632	train-auc:0.997077	valid-error:0.027558	valid-auc:0.997083
[200]	train-error:0.027643	train-auc:0.99708	valid-error:0.027558	valid-auc:0.997087
[201]	train-error:0.028846	train-auc:0.997069	valid-error:0.029374	valid-auc:0.997072
[202]	train-error:0.027683	train-auc:0.997093	valid-error:0.027665	valid-auc:0.997099
[203]	train-error:0.027615	train-auc:0.997119	valid-error:

[288]	train-error:0.021504	train-auc:0.997795	valid-error:0.021897	valid-auc:0.997752
[289]	train-error:0.021526	train-auc:0.997798	valid-error:0.021897	valid-auc:0.997757
[290]	train-error:0.021538	train-auc:0.997804	valid-error:0.021897	valid-auc:0.99776
[291]	train-error:0.021526	train-auc:0.997807	valid-error:0.02179	valid-auc:0.997764
[292]	train-error:0.021526	train-auc:0.99781	valid-error:0.02179	valid-auc:0.997767
[293]	train-error:0.021521	train-auc:0.997819	valid-error:0.02179	valid-auc:0.997774
[294]	train-error:0.021487	train-auc:0.997819	valid-error:0.02179	valid-auc:0.997775
[295]	train-error:0.021504	train-auc:0.997821	valid-error:0.021897	valid-auc:0.997777
[296]	train-error:0.02151	train-auc:0.997821	valid-error:0.021897	valid-auc:0.997782
[297]	train-error:0.02151	train-auc:0.997822	valid-error:0.02179	valid-auc:0.997784
[298]	train-error:0.02151	train-auc:0.997828	valid-error:0.021897	valid-auc:0.997794
[299]	train-error:0.02151	train-auc:0.997822	valid-error:0.02189

[384]	train-error:0.021251	train-auc:0.998129	valid-error:0.021683	valid-auc:0.998088
[385]	train-error:0.02124	train-auc:0.998132	valid-error:0.021683	valid-auc:0.998089
[386]	train-error:0.02124	train-auc:0.998133	valid-error:0.021683	valid-auc:0.998088
[387]	train-error:0.02124	train-auc:0.998186	valid-error:0.021683	valid-auc:0.998152
[388]	train-error:0.021223	train-auc:0.998188	valid-error:0.021577	valid-auc:0.998155
[389]	train-error:0.021223	train-auc:0.998189	valid-error:0.021577	valid-auc:0.99816
[390]	train-error:0.021217	train-auc:0.99819	valid-error:0.021577	valid-auc:0.998163
[391]	train-error:0.021217	train-auc:0.998194	valid-error:0.021577	valid-auc:0.998163
[392]	train-error:0.021212	train-auc:0.998197	valid-error:0.021577	valid-auc:0.998166
[393]	train-error:0.021217	train-auc:0.998198	valid-error:0.021577	valid-auc:0.998164
[394]	train-error:0.021189	train-auc:0.998208	valid-error:0.021577	valid-auc:0.998176
[395]	train-error:0.021212	train-auc:0.998224	valid-error:0

...

[2672]	train-error:0.006904	train-auc:0.999704	valid-error:0.008225	valid-auc:0.999549
[2673]	train-error:0.006909	train-auc:0.999704	valid-error:0.008225	valid-auc:0.999549
[2674]	train-error:0.006909	train-auc:0.999704	valid-error:0.008225	valid-auc:0.99955
[2675]	train-error:0.006909	train-auc:0.999704	valid-error:0.008225	valid-auc:0.99955
[2676]	train-error:0.006915	train-auc:0.999705	valid-error:0.008225	valid-auc:0.99955
[2677]	train-error:0.006921	train-auc:0.999705	valid-error:0.008225	valid-auc:0.99955
[2678]	train-error:0.006915	train-auc:0.999705	valid-error:0.008225	valid-auc:0.99955
[2679]	train-error:0.006898	train-auc:0.999705	valid-error:0.008225	valid-auc:0.99955
[2680]	train-error:0.006904	train-auc:0.999705	valid-error:0.008225	valid-auc:0.99955
[2681]	train-error:0.006893	train-auc:0.999705	valid-error:0.008225	valid-auc:0.99955
[2682]	train-error:0.006898	train-auc:0.999706	valid-error:0.008225	valid-auc:0.99955
[2683]	train-error:0.006898	train-auc:0.999706	valid

[2767]	train-error:0.004604	train-auc:0.999717	valid-error:0.005982	valid-auc:0.999557
[2768]	train-error:0.004604	train-auc:0.999717	valid-error:0.005982	valid-auc:0.999557
[2769]	train-error:0.004604	train-auc:0.999717	valid-error:0.005982	valid-auc:0.999557
[2770]	train-error:0.004604	train-auc:0.999717	valid-error:0.005982	valid-auc:0.999557
[2771]	train-error:0.004604	train-auc:0.999717	valid-error:0.005982	valid-auc:0.999557
[2772]	train-error:0.004604	train-auc:0.999717	valid-error:0.005982	valid-auc:0.999558
[2773]	train-error:0.004604	train-auc:0.999718	valid-error:0.005982	valid-auc:0.999558
[2774]	train-error:0.004599	train-auc:0.999718	valid-error:0.005982	valid-auc:0.999557
[2775]	train-error:0.004604	train-auc:0.999718	valid-error:0.005982	valid-auc:0.999557
[2776]	train-error:0.004604	train-auc:0.999718	valid-error:0.005982	valid-auc:0.999557
[2777]	train-error:0.004593	train-auc:0.999718	valid-error:0.005982	valid-auc:0.999557
[2778]	train-error:0.004593	train-auc:0.999

[2862]	train-error:0.004402	train-auc:0.99973	valid-error:0.005554	valid-auc:0.999566
[2863]	train-error:0.004402	train-auc:0.99973	valid-error:0.005554	valid-auc:0.999566
[2864]	train-error:0.004357	train-auc:0.99973	valid-error:0.005448	valid-auc:0.999566
[2865]	train-error:0.004357	train-auc:0.99973	valid-error:0.005448	valid-auc:0.999565
[2866]	train-error:0.004357	train-auc:0.99973	valid-error:0.005341	valid-auc:0.999565
[2867]	train-error:0.004357	train-auc:0.99973	valid-error:0.005341	valid-auc:0.999565
[2868]	train-error:0.004363	train-auc:0.999731	valid-error:0.005554	valid-auc:0.999566
[2869]	train-error:0.004363	train-auc:0.999731	valid-error:0.005554	valid-auc:0.999566
[2870]	train-error:0.004363	train-auc:0.999731	valid-error:0.005554	valid-auc:0.999566
[2871]	train-error:0.004357	train-auc:0.999731	valid-error:0.005554	valid-auc:0.999566
[2872]	train-error:0.004357	train-auc:0.999731	valid-error:0.005554	valid-auc:0.999566
[2873]	train-error:0.004351	train-auc:0.999732	va

[2957]	train-error:0.004256	train-auc:0.999742	valid-error:0.005127	valid-auc:0.999575
[2958]	train-error:0.004261	train-auc:0.999742	valid-error:0.005127	valid-auc:0.999575
[2959]	train-error:0.004256	train-auc:0.999742	valid-error:0.00502	valid-auc:0.999574
[2960]	train-error:0.004256	train-auc:0.999742	valid-error:0.00502	valid-auc:0.999574
[2961]	train-error:0.004256	train-auc:0.999742	valid-error:0.00502	valid-auc:0.999574
[2962]	train-error:0.00425	train-auc:0.999742	valid-error:0.00502	valid-auc:0.999575
[2963]	train-error:0.00425	train-auc:0.999742	valid-error:0.00502	valid-auc:0.999574
[2964]	train-error:0.00425	train-auc:0.999743	valid-error:0.00502	valid-auc:0.999573
[2965]	train-error:0.00425	train-auc:0.999742	valid-error:0.00502	valid-auc:0.999574
[2966]	train-error:0.00425	train-auc:0.999743	valid-error:0.00502	valid-auc:0.999573
[2967]	train-error:0.00425	train-auc:0.999743	valid-error:0.00502	valid-auc:0.999573
[2968]	train-error:0.00425	train-auc:0.999743	valid-error:

[3053]	train-error:0.004115	train-auc:0.999753	valid-error:0.005127	valid-auc:0.999579
[3054]	train-error:0.004104	train-auc:0.999753	valid-error:0.005127	valid-auc:0.99958
[3055]	train-error:0.004104	train-auc:0.999753	valid-error:0.005127	valid-auc:0.999579
[3056]	train-error:0.004104	train-auc:0.999753	valid-error:0.005127	valid-auc:0.999579
[3057]	train-error:0.004104	train-auc:0.999754	valid-error:0.005127	valid-auc:0.999579
[3058]	train-error:0.004104	train-auc:0.999754	valid-error:0.005127	valid-auc:0.999579
[3059]	train-error:0.004104	train-auc:0.999754	valid-error:0.005127	valid-auc:0.999579
[3060]	train-error:0.00411	train-auc:0.999754	valid-error:0.005127	valid-auc:0.999579
[3061]	train-error:0.004115	train-auc:0.999754	valid-error:0.005127	valid-auc:0.999579
[3062]	train-error:0.004121	train-auc:0.999754	valid-error:0.005127	valid-auc:0.999579
[3063]	train-error:0.004132	train-auc:0.999754	valid-error:0.005127	valid-auc:0.999579
[3064]	train-error:0.004121	train-auc:0.99975

In [102]:
print gbm.eval(dvalid)

[0]	eval-error:0.004807	eval-auc:0.999578


### Validate classification accuracy

In [103]:
%time data_ngrams = generate_context_ngrams(train, context_cols)

CPU times: user 11min 49s, sys: 1min 2s, total: 12min 52s
Wall time: 13min 12s


In [105]:
%time dtest = xgb.DMatrix(data_ngrams, label=labels, feature_names=feature_names)

CPU times: user 7min 20s, sys: 4min 41s, total: 12min 2s
Wall time: 14min 39s


In [106]:
%time predictions = gbm.predict(dtest)

CPU times: user 2h 12min 44s, sys: 31.5 s, total: 2h 13min 15s
Wall time: 47min 43s


In [107]:
predictions

array([ 0.01158588,  0.00036614,  0.01372064, ...,  0.0202615 ,
        0.00632452,  0.00073591], dtype=float32)

In [108]:
from sklearn.metrics import accuracy_score

for i in range(10):
    predictions_current = predictions > i / 10.0
    print ("0.{}: {}".format(i, accuracy_score(labels, predictions_current)))

0.0: 0.125082403416
0.1: 0.97505953236
0.2: 0.984837582407
0.3: 0.98892325983
0.4: 0.991354863932
0.5: 0.993797437818
0.6: 0.994522200149
0.7: 0.994743770888
0.8: 0.994721358483
0.9: 0.992554069454


In [109]:
best_threshold = 0.7
best_accuracy = accuracy_score(labels, predictions > best_threshold)
print 'Best accuracy: ', best_accuracy
print 'Number of errors: ', len(train) - best_accuracy * len(train)

Best accuracy:  0.994743770888
Number of errors:  55582.0


In [110]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(labels, predictions > best_threshold).ravel()

In [111]:
print tn, fp 
print fn, '\t', tp

9215552 36265
19317 	1303367
